# Querying Protests Worldwide
Requirements:
* ArcGIS API for Python
`conda install -c esri arcgis`
* arcpy or at least shapely for spatial joins


In [2]:
# author: Jan Tschada
# SPDX-License-Identifer: Apache-2.0

In [3]:
from arcgis.gis import GIS
from arcgis.features import FeatureSet
from arcgis.geometry.filters import intersects
from arcgis.mapping import generate_classbreaks
from datetime import datetime
from georapid.client import GeoRapidClient
from georapid.factory import EnvironmentClientFactory
from georapid.formats import OutFormat
from georapid.protests import aggregate
import pandas as pd

## Creating a client
---
**NOTE**

Ensure that `os.environ['x_rapidapi_key']` holds your Rapid API Key!

---

In [4]:
host = 'geoprotests.p.rapidapi.com'
client: GeoRapidClient = EnvironmentClientFactory.create_client_with_host(host)

## Connecting to ArcGIS Online

In [5]:
gis = GIS()

In [6]:
def generate_aggregated_renderer(spatial_df, column='count'):
    alpha = 0.35
    class_breaks_renderer = generate_classbreaks(spatial_df, geometry_type='Polygon', field=column, method='esriClassifyNaturalBreaks', class_count=5, colors='YlOrRd', alpha=0.35)
    class_breaks_renderer['defaultSymbol']['color'][3] = int(alpha * 255)
    for class_break_info in class_breaks_renderer['classBreakInfos']:
        class_break_info['symbol']['color'][3] = int(alpha * 255)
    return class_breaks_renderer

def plot_aggregated(map_view, spatial_df, column='count'):
    """
    Plots the spatial dataframe as classified polygons using the specified map view.
    """
    if spatial_df.empty:
        print("The dataframe is empty!")
    else:
        class_breaks_renderer = generate_aggregated_renderer(spatial_df)
        drawing_info = {
            'renderer': class_breaks_renderer
        }
        feature_collection = spatial_df.spatial.to_feature_collection(drawing_info=drawing_info)
        map_view.add_layer(feature_collection)
        
def query_aggregations(from_date, to_date):
    """
    Queries the specific date range and returns a data frame.
    """
    sdfs = []
    date_range = pd.date_range(start=from_date, end=to_date)
    for date_of_interest in date_range:
        aggregated_protests_dict = aggregate(client, date_of_interest, OutFormat.ESRI)
        aggregated_protests_fset = FeatureSet.from_dict(aggregated_protests_dict)
        aggregated_protests_sdf = aggregated_protests_fset.sdf
        if not aggregated_protests_sdf.empty:
            sdfs.append(aggregated_protests_sdf)
            
    if sdfs:
        return pd.concat(sdfs, ignore_index=True)
    else:
        return pd.DataFrame()
        
def query_world_countries(gis, two_digit_country_code):
    """
    Returns the geographic feature of a country.
    """
    world_countries_item_id = '2b93b06dc0dc4e809d3c8db5cb96ba69'
    world_countries_item = gis.content.get(world_countries_item_id)
    world_countries_layer = world_countries_item.layers[0]
    return world_countries_layer.query(where=f"ISO='{two_digit_country_code}'")

## Query the protests and try to narrow down those related to the French pension reform strikes

In [7]:
protest_map = gis.map('France')
protest_map.basemap = 'osm'
protest_map

MapView(layout=Layout(height='400px', width='100%'))

In [8]:
aggregated_protests_sdf = query_aggregations(datetime(2023, 1, 18), datetime(2023, 1, 20))
aggregated_protests_sdf.sort_values(by='count', ascending=False)

,OBJECTID,count,timestamp,Shape__Area,Shape__Length,SHAPE
107,38893,1028,2023-01-19,54126587736.526367,866025.403784,"{""rings"": [[[-8779178.09360229, -1337491.6572]..."
202,38988,722,2023-01-20,54126587736.526367,866025.403784,"{""rings"": [[[-8779178.09360229, -1337491.6572]..."
138,38924,366,2023-01-19,54126587736.527466,866025.403784,"{""rings"": [[[97582.2951882072, 6287508.3428], ..."
16,38802,261,2023-01-18,54126587736.526367,866025.403784,"{""rings"": [[[-8779178.09360229, -1337491.6572]..."
106,38892,204,2023-01-19,54126587736.527832,866025.403784,"{""rings"": [[[-8779178.09360229, -837491.657200..."
...,...,...,...,...,...,...
69,38855,1,2023-01-18,54126587736.527344,866025.403784,"{""rings"": [[[4860722.01600262, 4537508.3428], ..."
190,38976,1,2023-01-20,54126587736.527344,866025.403784,"{""rings"": [[[-11810267.0068478, 4912508.3428],..."
187,38973,1,2023-01-20,54126587736.53125,866025.403784,"{""rings"": [[[-13542317.8144167, 6412508.3428],..."
73,38859,1,2023-01-18,54126587736.527344,866025.403784,"{""rings"": [[[5077228.36694873, 3912508.3428], ..."


In [9]:
protest_map = gis.map('France')
protest_map.basemap = 'osm'
plot_aggregated(protest_map, aggregated_protests_sdf)
protest_map

MapView(layout=Layout(height='400px', width='100%'))

## Filter the news related to protests being located in France
---
**NOTE**

If your environment has no geometry engine (`arcpy` or `shapely`) the spatial join returns an empty data frame!

---

In [10]:
france_country_fset = query_world_countries(gis, two_digit_country_code='FR')
france_country_sdf = france_country_fset.sdf
france_country_sdf

,FID,COUNTRY,ISO,COUNTRYAFF,AFF_ISO,Shape__Area,Shape__Length,SHAPE
0,79,France,FR,France,FR,1162358729551.715332,8568685.86359,"{""rings"": [[[198339.953048288, 5246743.3982777..."


In [11]:
france_aggregated_protests_sdf = aggregated_protests_sdf.spatial.join(france_country_sdf[['COUNTRY', 'SHAPE']])
france_aggregated_protests_sdf = france_aggregated_protests_sdf.drop(columns=['index_right'])
france_aggregated_protests_sdf

,OBJECTID,count,timestamp,Shape__Area,Shape__Length,SHAPE,COUNTRY
0,38821,7,2023-01-18,54126587736.52771,866025.403784,"{""rings"": [[[-551936.757650122, 6162508.3428],...",France
1,38823,1,2023-01-18,54126587736.527405,866025.403784,"{""rings"": [[[-335430.406704012, 6287508.3428],...",France
2,38824,6,2023-01-18,54126587736.527344,866025.403784,"{""rings"": [[[-335430.406704012, 5787508.3428],...",France
3,38826,7,2023-01-18,54126587736.527405,866025.403784,"{""rings"": [[[-118924.055757903, 5662508.3428],...",France
4,38827,2,2023-01-18,54126587736.527344,866025.403784,"{""rings"": [[[-118924.055757903, 5412508.3428],...",France
5,38828,6,2023-01-18,54126587736.527283,866025.403784,"{""rings"": [[[97582.2951882072, 6537508.3428], ...",France
6,38829,36,2023-01-18,54126587736.527466,866025.403784,"{""rings"": [[[97582.2951882072, 6287508.3428], ...",France
7,38832,5,2023-01-18,54126587736.527466,866025.403784,"{""rings"": [[[314088.646134317, 5912508.3428], ...",France
8,38834,2,2023-01-18,54126587736.527344,866025.403784,"{""rings"": [[[747101.348026536, 6412508.3428], ...",France
9,38835,7,2023-01-18,54126587736.526855,866025.403784,"{""rings"": [[[747101.348026536, 5912508.3428], ...",France


In [12]:
protest_map = gis.map('France')
protest_map.basemap = 'osm'
plot_aggregated(protest_map, france_aggregated_protests_sdf)
protest_map

MapView(layout=Layout(height='400px', width='100%'))